In [5]:
# --- 1. 라이브러리 설치 및 임포트 ---

# .env 파일을 사용하기 위한 라이브러리를 설치합니다.
!pip install python-dotenv

import pandas as pd
import requests
import time
from tqdm import tqdm
import os
from dotenv import load_dotenv # .env 파일을 읽기 위한 라이브러리

# tqdm 라이브러리와 pandas의 연동을 설정합니다.
tqdm.pandas()

In [20]:
# --- 2. 오류 메시지 출력 함수 ---
def print_error(text):
    # ANSI 이스케이프 코드: '\033[91m'는 빨간색, '\033[0m'는 색상 리셋
    print(f"\033[91m{text}\033[0m")

In [ ]:
# --- 3. .env 파일에서 API 키 로드 ---

# .env 파일에 정의된 환경 변수를 현재 세션으로 로드합니다.
load_dotenv()

# API 키 관리를 위한 전역 변수
key_idx = 0
all_keys_exhausted = False

# KAKAO_REST_API_KEY1부터 KAKAO_REST_API_KEY10까지의 키를 저장할 리스트
KAKAO_KEYS = []

# 여러 개의 API 키를 리스트로 불러오기
for i in range(1, 11):  # 최대 10개 지원 (필요시 숫자 변경)
    # os.getenv() 함수를 사용하여 "KAKAO_REST_API_KEY" 라는 이름의 환경 변수 값을 가져옵니다.
    key = os.getenv(f"KAKAO_REST_API_KEY{i}")
    if key: KAKAO_KEYS.append(key)

if not KAKAO_KEYS:
    print_error("[오류] '.env 파일에 KAKAO_REST_API_KEY1~N 형식으로 키가 필요합니다.")
else:
    print(f"총 {len(KAKAO_KEYS)}개의 API 키 로드 완료.")

총 10개의 API 키 로드 완료.


In [ ]:
# --- 4. 경로 설정 및 데이터 로드 ---

# 원본 학습 데이터 경로 지정
TRAIN_DATA_PATH = '../data/processed/train_geocoded.csv'

# 처리된 데이터가 저장될 디렉토리 경로
OUTPUT_DIR = '../data/processed/'

# 도로명 주소가 없는 행을 저장할 파일명
NO_ADDRESS_FILENAME = 'missing_address_rows.csv'

# 중간 저장 체크포인트 파일명
CHECKPOINT_FILENAME = 'train_geocoded_checkpoint.csv'

# 최종 결과 파일명
FINAL_FILENAME = 'train_geocoded.csv'

# 각 파일의 전체 경로 구성
NO_ADDRESS_PATH = os.path.join(OUTPUT_DIR, NO_ADDRESS_FILENAME)
CHECKPOINT_PATH = os.path.join(OUTPUT_DIR, CHECKPOINT_FILENAME)
FINAL_PATH = os.path.join(OUTPUT_DIR, FINAL_FILENAME)

# 데이터 로드 시작 메시지 출력
print(f"'{TRAIN_DATA_PATH}'에서 원본 데이터를 로드합니다...")

try:
    # 원본 CSV 파일을 pandas로 읽어옴
    train_df = pd.read_csv(TRAIN_DATA_PATH, low_memory=False)
    print("데이터 로드 완료.")
except FileNotFoundError:
    # 파일이 존재하지 않을 경우 예외 처리
    print_error(f"오류: ''except FileNotFoundError:' / '{TRAIN_DATA_PATH}' 파일을 찾을 수 없습니다.")
    train_df = None

'../data/raw/train.csv'에서 원본 데이터를 로드합니다...
데이터 로드 완료.


In [35]:
# --- 5. 지오코딩 및 주소 생성 함수 정의 ---

# 주어진 주소와 API 키 인덱스를 이용해 좌표(X, Y)를 요청
def get_coordinates(address, key_index):
    # 현재 사용할 카카오 API 키 선택
    api_key = KAKAO_KEYS[key_index]
    
    # 카카오 주소 검색 API URL 구성
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={address}"
    
    # 요청 헤더에 API 키 추가
    headers = {"Authorization": f"KakaoAK {api_key}"}

    try:
        # API에 GET 요청을 보내고 응답을 받음
        response = requests.get(url, headers=headers)
        
        # 응답 상태 코드가 200(성공)이 아닐 경우 예외 발생
        response.raise_for_status()
        
        # JSON 형식으로 응답 데이터 파싱
        data = response.json()

        # 주소 정보가 존재할 경우 첫 번째 결과의 좌표 반환
        if data['documents']:
            return float(data['documents'][0]['x']), float(data['documents'][0]['y'])
        
        # 주소 정보가 없을 경우 None 반환
        else:
            # [추가된 부분] API 검색 결과가 없는 경우 원인 출력
            print_error('def get_coordinates(address, key_index): 주소 정보 존재여부 체크')
            print_error(f"  [실패] API 검색 결과 없음: '{address}'")
            return None

    # HTTP 오류가 발생했을 경우
    except requests.exceptions.HTTPError as e:
        # API 사용량 초과(429) 또는 권한 없음(403)일 경우 키 교체 지시
        if e.response.status_code == 429 or e.response.status_code == 403:
            return "CHANGE_KEY"
        
        print_error('def get_coordinates(address, key_index): / except requests.exceptions.HTTPError')
        print(f"  [실패] API 요청 실패 (HTTP 오류): {e}, 주소: '{address}'")
        
        # 기타 오류는 None 반환
        return None
    
    except (requests.exceptions.ConnectionError, requests.exceptions.Timeout):
        # 네트워크 오류 발생 시 메시지 출력 후 재시도 지시
        print_error('def get_coordinates(address, key_index): / except (requests.exceptions.ConnectionError, requests.exceptions.Timeout)')
        print_error(f"[네트워크 오류] 인터넷 연결이 끊겼거나 응답 시간이 초과되었습니다. 주소: '{address}'")
        time.sleep(5)  # 잠시 대기
        return "RETRY"
    
    # 그 외 예외 상황도 모두 None 반환
    except Exception as e:
        print_error('5. 지오코딩 및 주소 생성 함수 정의')
        print_error('except Exception as e:')
        print(f"  [실패] 알 수 없는 오류 발생: {e}, 주소: '{address}'")
        return None  

# 여러 API 키를 순환하며 지오코딩을 시도하는 함수
def geocode_with_key_rotation(address):
    # 전역 변수로 선언된 key_idx 사용
    global key_idx, all_keys_exhausted
    
    # 주소가 결측치일 경우 None 반환
    if pd.isnull(address):
        return None

    # 현재 키 인덱스를 저장 (한 바퀴 돌았는지 확인용)
    start_key_idx = key_idx
    
    # 네트워크 오류 재시도 횟수 초기화
    retry_attempts = 0

    while True:
        # 현재 키로 지오코딩 시도
        result = get_coordinates(address, key_idx)

        # API 키 변경이 필요한 경우
        if result == "CHANGE_KEY":
            # 다음 키로 인덱스 전환
            key_idx = (key_idx + 1) % len(KAKAO_KEYS)
            
            # 한 바퀴를 다 돌았으면 모든 키 소진 처리
            if key_idx == start_key_idx:
                all_keys_exhausted = True
                return None
            
            # 다음 키로 요청하기 전 잠시 대기
            time.sleep(1)
            continue
        
        elif result == "RETRY":
            # 네트워크 오류 재시도
            retry_attempts += 1
            
            # 재시도 횟수가 5회를 초과하면 오류 메시지 출력 후 None 반환
            if retry_attempts >= 5:
                print_error(f"[재시도 실패] 주소: '{address}' - 네트워크 오류가 계속 발생하여 건너뜁니다.")
                return None
            time.sleep(5)
            continue


        # 좌표가 정상적으로 반환된 경우
        return result

In [36]:
# --- 6. 주소 생성 함수 정의 (도로명/번지 예외 처리) ---

def create_full_address(row):
    """
    도로명 주소를 우선 사용하되, 없으면 번지 주소를 사용합니다.
    """
    # '도로명'이 유효한 문자열인지 확인
    if isinstance(row['도로명'], str) and row['도로명'].strip():
        return row['시군구'] + ' ' + row['도로명']
    # '도로명'이 없으면 '번지'가 유효한지 확인
    elif isinstance(row['번지'], str) and row['번지'].strip():
        return row['시군구'] + ' ' + row['번지']
    # 둘 다 없으면 None 반환
    else:
        return None

In [37]:
# --- 7. 결측치 채우기 실행 ---

# 데이터프레임이 정상적으로 로드되었고, API 키가 존재하는 경우에만 실행
if train_df is not None and KAKAO_KEYS:
    
    # 좌표가 비어있는 행만 복사하여 작업 대상으로 지정
    df_to_geocode = train_df[train_df['좌표X'].isnull()].copy()
    
    if not df_to_geocode.empty:
        print(f"\n총 {len(df_to_geocode)}개의 좌표 결측치에 대한 작업을 시작합니다...")

        # 7.1. 주소 생성 및 주소 없는 행 분리/저장
        # 도로명 또는 번지를 기준으로 전체 주소를 생성하고
        # full_address가 없는 행은 별도로 저장
        df_to_geocode['full_address'] = df_to_geocode.apply(create_full_address, axis=1)
        no_address_rows = df_to_geocode[df_to_geocode['full_address'].isnull()]

        if not no_address_rows.empty:
            # 저장 디렉토리가 없으면 생성
            os.makedirs(OUTPUT_DIR, exist_ok=True)
            # 도로명/번지 모두 없는 행 저장
            no_address_rows.to_csv(NO_ADDRESS_PATH, index=False)
            print(f"{len(no_address_rows)}개의 주소 없는 행을 '{NO_ADDRESS_PATH}'에 저장했습니다.")
        else:
            print("모든 행에 '도로명' 또는 '번지' 주소 정보가 존재합니다.")

        # 주소가 존재하는 행만 남기고 필터링
        df_to_geocode.dropna(subset=['full_address'], inplace=True)

        # 7.2. 고유 주소에 대한 API 호출 (캐싱)
        # 중복된 주소는 API 호출을 1회만 하기 위해 고유한 주소만 추출
        unique_addresses = df_to_geocode['full_address'].unique()
        print(f"실제 API 호출 대상 고유 주소 개수: {len(unique_addresses)}")
        
        # 주소별 좌표를 저장할 캐시 딕셔너리
        address_cache = {}
        processed_count = 0  # 처리된 주소 수 카운트

        # 고유 주소 목록을 순회하며 지오코딩 수행
        for addr in tqdm(unique_addresses, desc="Geocoding Progress"):
            # 모든 키가 소진되었으면 루프 중단
            if all_keys_exhausted:
                print_error("\n!! 모든 키 사용량 소진: 작업 중단 !!")
                break
            
            # 현재 주소에 대해 좌표 요청
            coords = geocode_with_key_rotation(addr)

            # 좌표를 성공적으로 받아온 경우 캐시에 저장
            if coords:
                address_cache[addr] = coords
                processed_count += 1
            
            # 7.3. 1000개 단위로 임시 저장 (체크포인트)
            if processed_count > 0 and processed_count % 1000 == 0:
                print(f"\n--- {processed_count}개 처리 완료. 체크포인트를 저장합니다 ---")
                
                # 원본 데이터 복사 후 full_address 다시 생성
                temp_df = train_df.copy()
                temp_df['full_address_map'] = temp_df.apply(create_full_address, axis=1)
                
                # 캐시된 주소를 기반으로 좌표 매핑
                temp_coords = temp_df['full_address_map'].map(address_cache)
                valid_coords_map = temp_coords.dropna()

                # 좌표 값 반영
                temp_df.loc[valid_coords_map.index, '좌표X'] = [x[0] for x in valid_coords_map]
                temp_df.loc[valid_coords_map.index, '좌표Y'] = [x[1] for x in valid_coords_map]

                # 임시 컬럼 제거 후 저장
                temp_df.drop(columns=['full_address_map'], inplace=True)
                temp_df.to_csv(CHECKPOINT_PATH, index=False)
                print(f"체크포인트 저장 완료: '{CHECKPOINT_PATH}'")

        # 7.4. 캐싱된 결과 매핑 및 최종 업데이트
        print("\n최종 결과 업데이트를 시작합니다...")

        # df_to_geocode의 주소를 기준으로 캐시된 좌표 매핑
        final_coords_map = df_to_geocode['full_address'].map(address_cache)
        valid_final_coords = final_coords_map.dropna()

        # 원본 train_df의 해당 인덱스에 좌표 반영
        train_df.loc[valid_final_coords.index, '좌표X'] = [x[0] for x in valid_final_coords]
        train_df.loc[valid_final_coords.index, '좌표Y'] = [x[1] for x in valid_final_coords]
        print("결측치 업데이트 완료!")

    else:
        print("\n좌표에 결측치가 없어 추가 작업을 진행하지 않습니다.")


총 869670개의 좌표 결측치에 대한 작업을 시작합니다...
모든 행에 '도로명' 또는 '번지' 주소 정보가 존재합니다.
실제 API 호출 대상 고유 주소 개수: 8504


Geocoding Progress:   0%|          | 11/8504 [00:10<51:08,  2.77it/s]  

def get_coordinates(address, key_index): 주소 정보 존재여부 체크
  [실패] API 검색 결과 없음: '서울특별시 강남구 개포동 언주로 21'


Geocoding Progress:   2%|▏         | 133/8504 [00:21<23:00,  6.07it/s]

def get_coordinates(address, key_index): 주소 정보 존재여부 체크
  [실패] API 검색 결과 없음: '서울특별시 강남구 도곡동 남부순환로 2803'


...

def get_coordinates(address, key_index): 주소 정보 존재여부 체크
  [실패] API 검색 결과 없음: '서울특별시 강남구 대치동 633'
def get_coordinates(address, key_index): 주소 정보 존재여부 체크
  [실패] API 검색 결과 없음: '서울특별시 강남구 역삼동 763'


Geocoding Progress: 100%|█████████▉| 8503/8504 [13:56<00:00, 13.78it/s]

def get_coordinates(address, key_index): 주소 정보 존재여부 체크
  [실패] API 검색 결과 없음: '서울특별시 서초구 서초동 1686-4'
def get_coordinates(address, key_index): 주소 정보 존재여부 체크
  [실패] API 검색 결과 없음: '서울특별시 서초구 잠원동 신반포로 45'


Geocoding Progress: 100%|██████████| 8504/8504 [13:56<00:00, 10.16it/s]


def get_coordinates(address, key_index): 주소 정보 존재여부 체크
  [실패] API 검색 결과 없음: '서울특별시 용산구 한강로2가 196'

최종 결과 업데이트를 시작합니다...
결측치 업데이트 완료!


In [38]:
# --- 8. 결과 확인 및 저장 ---

# train_df가 정상적으로 존재할 경우에만 저장 진행
if train_df is not None:
    # 좌표X의 남은 결측치 개수를 최종 확인
    final_missing_count = train_df['좌표X'].isnull().sum()
    print(f"\n작업 후 남은 '좌표X'의 결측치 개수: {final_missing_count}")

    # 저장 디렉토리가 없을 경우 생성
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 전체 train_df를 최종 결과 파일로 저장
    train_df.to_csv(FINAL_PATH, index=False)
    print(f"모든 작업이 완료되었습니다. 최종 데이터가 '{FINAL_PATH}' 경로에 저장되었습니다.")

else:
    # train_df가 None이면 에러 메시지 출력 후 저장하지 않음
    print_error("\n[작업 중단] 데이터 파일이 없어 최종 저장을 진행할 수 없습니다.")


작업 후 남은 '좌표X'의 결측치 개수: 22227
모든 작업이 완료되었습니다. 최종 데이터가 '../data/processed/train_geocoded.csv' 경로에 저장되었습니다.
